In [1]:
from urllib.request import urlretrieve
import jieba
import warnings
warnings.filterwarnings("ignore")
"""
url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
urlretrieve(url, "dict.big")
url = "https://github.com/Elwing-Chou/mtkdl/raw/master/ptt_dic.txt"
urlretrieve(url, "dict.ptt")
"""
jieba.set_dictionary("dict.big")
jieba.load_userdict("dict.ptt")

Building prefix dict from D:\onedrive\Documents\data\dict.big ...
Loading model from cache R:\TEMP\jieba.u3fc4963e5b0b71d0127ab7586a4086cb.cache
Loading model cost 1.364 seconds.
Prefix dict has been built successfully.


In [2]:
import pandas as pd
data = pd.read_csv("data.csv", header=0)

In [3]:
df = data.copy()
df["combine"] = df[df.columns[5:]].apply(" ".join, axis=1)

In [4]:
import re
def process(content):
    content = re.sub(r'https?:\/\/.*[\r\n]*', '', content)
    punct = set(u''':!),.:;?]}¢'"、。〉》」』】〕〗〞︰︱︳﹐､﹒﹔﹕﹖﹗﹚﹜﹞！），．：；？｜｝︴︶︸︺︼︾﹀﹂﹄﹏､～￠々‖•·ˇˉ―--′’”([{£¥'"‵〈《「『【〔〖（［｛￡￥〝︵︷︹︻︽︿﹁﹃﹙﹛﹝（｛“‘-—_…~/ －＊➜■─★☆=@<>◉é''')
    cut = filter(lambda x: x not in punct, jieba.cut(content))
    return " ".join(cut)
df["combine"] = df["combine"].apply(process)
df["combine"] = df["combine"].str.replace("NONE", "")
df["combine"] = df["combine"].str.replace("的", "")
df["combine"] = df["combine"].str.replace("我", "")
df["combine"] = df["combine"].str.replace("是", "")
df["combine"] = df["combine"].str.replace("買", "")
df["combine"] = df["combine"].str.replace("了", "")
df["combine"] = df["combine"].str.replace("有", "")
df["combine"] = df["combine"].str.replace("很", "")

In [5]:
with open("lines.txt", "w", encoding="utf-8") as f:
    for content in df["combine"]:
        f.write(content)
        f.write("\n")

In [6]:
from gensim.models.word2vec import LineSentence
from gensim.models.word2vec import Word2Vec
sentences = LineSentence("lines.txt")
model = Word2Vec(sentences)

In [7]:
vocab = "武士刀"
model.wv[vocab]
model.wv.most_similar(vocab)

[('PONY', 0.9985829591751099),
 ('推推', 0.9982659816741943),
 ('銷售', 0.9982307553291321),
 ('時代', 0.9981347322463989),
 ('此款', 0.9980760812759399),
 ('早已', 0.9980430006980896),
 ('蘭芝', 0.9979372024536133),
 ('替換', 0.9979318976402283),
 ('必', 0.9978307485580444),
 ('MEKO', 0.9978154301643372)]

In [8]:
import numpy as np
import jieba.analyse
encodings = []
for content in df["combine"]:
    tags = jieba.analyse.extract_tags(
        content, 
        topK=None, 
        withWeight=True
    )
    total = np.zeros((100, ))
    weight = 0.0
    for word, tfidf in tags:
        if word in model.wv.vocab:
            total = total + tfidf * model.wv[word]
            weight = weight + tfidf
    encodings.append(total / weight)
print(len(encodings))

867


In [11]:
with open("w2v_vocab.wv", "w", encoding="utf-8") as f:
    f.write("{} {}\n".format(867, 100))
    for i, wv in enumerate(encodings):
        wv = " ".join(map(str, wv))
        f.write("{} {}\n".format(i, wv))

In [12]:
from gensim.models import KeyedVectors
d2v = KeyedVectors.load_word2vec_format("w2v_vocab.wv")

In [13]:
tok = input("Enter(0-866):")
data = df.iloc[int(tok)]
print("產品名", data["產品名"])
print("內容:", data["combine"])
print("-" * 30)
tok2, sim = d2v.most_similar(tok)[0]
print("相似度:", sim)
data = df.iloc[int(tok2)]
print("產品名", data["產品名"])
print("內容:", data["combine"])

Enter(0-866):20
產品名 久耀閃閃打亮盤
內容: 忘   被 版 上 哪一位 網友 給 生火  她 刷 這顆  光澤 讓  好燒 啊 但  對 打亮 最 不能 接受  就 黃 光 而 最 多 網友 開箱  炫光金 就 帶 一點 黃 光 黃皮 人 真 超怕 黃 上加 黃 後來 才 發現 1028 根本 就  出 偏 白光 版  銀 白金 不 知道 到底  他 缺貨 太久 還  一直 眼殘 沒 看到 總之 白 金色 極合 意 就 趁 週六 77 折帶  之前 沒  到 限量 盤 殘念 好 在 1028 又 出  這款 久耀 閃閃 打亮 盤 ❤ 這款 只 單一 色 炫光金 而且 這次  壓紋 超美 ❤ ❤ ❤ ❤  鑽石 光芒  壓紋 光看 心情 就  好 XD 一盒  8g 可以 用到 天荒地老 特價 時 不到 三百元 CP 值  高 ❤ 顏色  炫光金 粉質 算細 不太會 飛粉 之前 1028 出  打亮 餅 都 沒  到 當 在 寶雅 逛  時候  看到  這顆 打亮  以 為  重新 回歸 後來 才 發現 原來  新 產品 這顆 花紋 做 得 漂亮 手背 刷色 摸起來 點 諾諾  刷色 看起來 也  超閃  來看 妝效  沒  到 之前  打亮 餅 所以  也 不 清楚 差 在 哪 其實 少 會 特別注意 開架  打亮 產品 因為 catie  推薦 過 網路上 討論 度 也 蠻高  看到 它  特價 想 說 來 試 看看 而且 容量 跟 粉餅 一樣 足足 8G 掀開 後 鑽石 般 切割 面 閃閃發亮 顏色 為 香檳金 質地 柔滑 金屬 光  光澤感 閃耀 光芒 用 指腹 輕輕 一抹 亮度高 提亮  效果 好 強烈  金屬 光感 讓 五官 更加 立體 終於 被  等到  炫光金 💫 🌟 ✨ 沒想到  也 會 成為 打亮 控  一員 🤩 之前 就 看到 多 人 推薦 1028  打亮 那 時候  2 個 顏色 炫光金 & 暮光橘 不過  去 找  時候 都 只 剩下 暮光橘 顏色  比較 健康 膚色  所以 一直  糾結 到底 要 不要 下手 還好  忍住  😁 😁 這次  名稱 變成 # 1028 久耀 閃閃 打亮 盤 最近 媽咪 再練 手感 想到 就 會 化妝 主要  近視 真 太過 深  眼鏡 拿掉 幾乎 看不見 偏偏 又 無法 戴 隱形眼鏡 所